In [1]:
import pandas as pd
from web3 import Web3
from web3.middleware import geth_poa_middleware

In [2]:
chain = 'eth'
query_mode = 'http'
w3 = None
data = {
    'eth': {
        'rpc': '/eth/eth_node/node/geth.ipc',
        'http': 'https://mainnet.infura.io/v3/e0a4e987f3ff4f4fa9aa21bb08f09ef5',
        'datafile': 'data/one_day_exit_scam_eth.csv',
        'edgefile': 'data/tx_eth.csv',
        'nodefile': 'data/address_eth.csv'
    },
    'bsc': {
        'rpc': '',
        'http': 'https://bsc-dataseed.binance.org/',
        'datafile': 'one_day_exit_scam_bsc.csv',
        'edgefile': 'data/tx_bsc.csv',
        'nodefile': 'data/address_bsc.csv'
    }
}

In [3]:
def get_w3():
    tmp = data[chain][query_mode]
    w3_data = {
        'eth': {
            'rpc': Web3(Web3.IPCProvider(tmp)),
            'http': Web3(Web3.HTTPProvider(tmp))
        },
        'bsc': {
            'rpc': None,
            'http': Web3(Web3.HTTPProvider(tmp)).middleware_onion.inject(geth_poa_middleware, layer=0)
        } 
    }
    return w3_data[chain][query_mode]

In [4]:
w3 = get_w3()

In [5]:
address_df = pd.read_csv(data[chain]['nodefile'])
transaction_df = pd.read_csv(data[chain]['edgefile'])

In [ ]:
tmp = transaction_df.copy()
for index, row in tmp.iterrows():
    if row['status'] == 'None':
        tx = w3.eth.get_transaction_receipt(row['hash'])
        transaction_df.at[index, 'status'] = tx['status']
    if index % 300 == 0:
        transaction_df.to_csv(str(index)+data[chain]['edgefile'], index=False)
transaction_df.to_csv(data[chain]['edgefile'], index=False)